### This notebook can read a TKD txt file contining exit energies and depths and plot energy-depth KDE distributions 

In [13]:
import sys
import h5py
import pylab

import numpy as np
 
import plotly.plotly as py
import plotly.graph_objs as go

import scipy.stats as st      
from scipy import interpolate

#### Read the energy data from the file

In [3]:
energy_100 = []
depth_100 = []

fileName100 = 'TKDpaper/txt_files/TKD100_10000.txt'

with open(fileName100, "r") as data:
    for line in data: #read one line at a time
        column = line.replace(':',' ').replace(',',' ').split()
        energy_100.append(float(column[0]))
        depth_100.append(float(column[1]))

In [4]:
energy_200 = []
depth_200 = []

fileName200 = 'TKDpaper/txt_files/TKD200_10000.txt'

with open(fileName200, "r") as data200:
    for line in data200: #read one line at a time
        if ('Total' not in line) and ('Number' not in line): 
            column = line.replace(':',' ').replace(',',' ').split()
            energy_200.append(float(column[0]))
            depth_200.append(float(column[1]))


#### KDE function and 90% line

In [14]:
def kde_scipy( vals1, vals2, (a,b), (c,d), N, bw ):
    x = np.linspace(a,b,N)
    y = np.linspace(c,d,N)
    X,Y = np.meshgrid(x,y)
    positions = np.vstack([Y.ravel(), X.ravel()])

    values = np.vstack([vals1, vals2])
    kernel = st.gaussian_kde(values, bw_method = bw)
    Z = np.reshape(kernel(positions).T, X.shape)

    return [x, y, Z]


def make_kdetrace(varX, varY, (a,b), (c,d), N, bw, colorsc):
    x, y, Z = kde_scipy(varY, varX, (a,b), (c,d), N, bw )

    trace = go.Contour(
           z=Z,
           x=x,
           y=y,
           colorscale=colorsc,
           #reversescale=True,
           opacity=0.9,
           ncontours=20,
           contours=dict(
                coloring='fill', 
                showlines=False, 
                start=0.001, 
                end = 0.5
           ), 
            line=dict(
                width=0.5, 
                   ), 
            zauto=False,
            zmax=0.5
           
       )

    
    return trace


def make_kdePercLines(varX, varY, (a,b), (c,d), N, bw, colorsc):
    
    x, y, Z = kde_scipy(varY, varX, (a,b), (c,d), N, bw )

    # show percentage contours
    Zp = Z / Z.sum()
    
    n = 100
    t = np.linspace(0, Zp.max(), n)
    integral = ((Zp >= t[:, None, None]) * Zp).sum(axis=(1,2))

    f = interpolate.interp1d(integral, t)
    t_contours = f(np.array([0.9, 0.8, 0.7, 0.6, 0.5, 0.4])) # t_contours are the margins of an area containing a percentage of the points

    traces = dict()
    for index, t in enumerate(t_contours):
        # one percentage contour at a time
        traces[str(index)] = go.Contour(
                   x=x,
                   y=y,
                   z=Zp,
                   opacity=0.5,
                   showlegend=False,
                   autocontour=False,
                   autocolorscale=False,
                   colorscale=colorsc,
                   ncontours=1,
                   contours=dict(
                       coloring='lines',
                       
                       start=t_contours[index], 
                       end = t_contours[index]+1
                   ),
                   line=dict(
                        width=2.5, 
                   ), 
                   showscale=False
               )


    return traces


#### Define a color scheme

In [15]:
purples=[[0.0,   '#fcf9f7'],
 [0.1666666666666666, '#edcfc9'],
 [0.3333333333333333, '#daa2ac'],
 [0.5,                '#bc7897'],
 [0.6666666666666666, '#925684'],
 [0.8333333333333333, '#5f3868'],
 [1.0,                '#2d1e3e']]

In [ ]:
x1, x2 = (0, 25)  # depth range
y1, y2 = (16, 20) # energy range

N = 200  # point to sample on a mesh for the KDE
bw = 0.3 # bandwidth

#### KDE for 100 nm film 

In [17]:
P_contours = make_kdePercLines(depth_100, energy_100, (x1, x2), (y1, y2), N, bw, 'Greys')
KDEtrace = make_kdetrace(depth_100, energy_100, (x1, x2), (y1, y2), N, bw, purples)

layoutKDE =  go.Layout(xaxis = dict(title = 'escape depth (nm)'), 
                    yaxis = dict(title = 'escape energy (keV)'),
                    title = '100nm thin film sample',
                    showlegend=False,   
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)')


layout = go.Layout(
    autosize=False,
    width=450,
    height=380,
    showlegend=False, 
    xaxis=dict(
        titlefont=dict(
          family='Times New Roman',
          size=14
        ),
        range=[0, 25],
        title='<b>Escape distance (mm)<b>', 
        mirror="ticks",
        showgrid=False,
        showline=True,
        ticks='inside',
        ticktext=['<b>0<b>','<b>5<b>', '<b>10<b>', '<b>15<b>', '<b>20<b>', '<b>25<b>' ], 
        tickvals=[0, 5, 10, 15, 20, 25],
        zeroline=False,
        tickfont=dict(
          family='Times New Roman',
          size=14
        ),
    ),
    yaxis=dict(
        titlefont=dict(
        family='Times New Roman',
        size=14
        ),
        range=[16, 20],
        #tickvals = [0, 1],
        mirror="ticks",
        showgrid=False,
        showline=True,
        title='<b>Escape energy (keV) <b>', 
        ticks='inside',
        ticktext=['<b>16<b>','<b>17<b>', '<b>18<b>', '<b>19<b>', '<b>20<b>'], 
        tickvals=[16, 17, 18, 19, 20],
        tickfont=dict(
          family='Times New Roman',
          size=14
        ),
    ),
    legend=dict(
          x=.1, y=0.3,
          font= dict(
              family='Times New Roman',
              size=16)
     ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)    


data = [ KDEtrace, P_contours[str(0)]]
figureKDE = go.Figure(data=data, layout= layout)

py.iplot(figureKDE)
#py.image.save_as(figureKDE, filename='TKDpaper/TKD100KDE.svg')

#### KDE for 200 nm film 


In [18]:
P_contours = make_kdePercLines(depth_200, energy_200, (x1, x2), (y1, y2), N, bw, 'Greys')
KDEtrace = make_kdetrace(depth_200, energy_200, (x1, x2), (y1, y2), N, bw, purples)

data = [ KDEtrace, P_contours[str(0)]]
figureKDE = go.Figure(data=data, layout= layout)


py.iplot(figureKDE)
#py.image.save_as(figureKDE, filename='TKDpaper/TKD200KDE.svg')
